In [2]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp


2024-11-14 14:34:58.071892: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#clase de configuracion del media pipe para detectar manos
class detectorManos():
    def __init__(self, mode=False, maxManos = 2, model_complexity=0, Confdeteccion = 0.5, Confsegui = 0.5):
        self.mode = mode
        self.maxManos = maxManos
        self.model_complexity = model_complexity
        self.Confdeteccion = Confdeteccion
        self.Confsegui = Confsegui

        self.mpmanos = mp.solutions.hands
        self.manos = self.mpmanos.Hands(self.mode, self.maxManos, self.model_complexity, self.Confdeteccion, self.Confsegui)
        self.dibujo = mp.solutions.drawing_utils
        self.tip = [4, 8, 12, 16, 20]

    def encontrarManos(self, frame, dibujar = False):
        imgcolor = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        self.resultados = self.manos.process(imgcolor)

        if self.resultados.multi_hand_landmarks:
            for mano in self.resultados.multi_hand_landmarks:
                if dibujar:
                    self.dibujo.draw_landmarks(frame, mano, self.mpmanos.HAND_CONNECTIONS)
        return frame
    
    def encontrarPosicion(self, frame, ManoNum = 0, dibujar = False):
        xlista = []
        ylista = []
        bbox = []
        self.lista = []
        if self.resultados.multi_hand_landmarks:
            mi_mano = self.resultados.multi_hand_landmarks[ManoNum]
            for id, lm in enumerate(mi_mano.landmark):
                alto, ancho, c = frame.shape
                cx, cy = int(lm.x * ancho), int(lm.y * alto)
                xlista.append(cx)
                ylista.append(cy)
                self.lista.append([id, cx, cy])
                if dibujar:
                    cv2.circle(frame, (cx, cy), 5, (0, 0, 0), cv2.FILLED)
            xmin, xmax = min(xlista), max(xlista)
            ymin, ymax = min(ylista), max(ylista)
            bbox = xmin, ymin, xmax, ymax
            if dibujar:
                cv2.rectangle(frame, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20), (0, 255, 0), 2)
        return self.lista, bbox
    
    def dedosArriba(self):
        dedos = []
        if self.lista[self.tip[0]][1] < self.lista[self.tip[0]-1][1]:
            dedos.append(1)
        else:
            dedos.append(0)
        
        for id in range(1,5): 
            if self.lista[self.tip[id]][2] < self.lista[self.tip[id]-2][2]:
                dedos.append(1)
            else:
                dedos.append(0)
        return dedos
    
    def numeroDedos(self):
        if(len(self.lista) != 0):
            dedos = self.dedosArriba()
            return sum(dedos)

In [ ]:
import warnings

detector = detectorManos(0.75)
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database') # Ignorar advertencia de protobuf

# Inicializar MediaPipe Face Mesh para detectar los puntos de la cara
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.2)
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.2)

# Iniciar la captura de video
vid = cv2.VideoCapture(0)

flag=0

while(True):
    
    ret, frame = vid.read()
    if not ret:
            break
    
    frame = detector.encontrarManos(frame)
    manosInfo, cuadro = detector.encontrarPosicion(frame, dibujar=False)
    
    if(detector.numeroDedos()== 1):
        # Cargar la imagen de las gafas
        img = cv2.imread('gafas1.jpg')
        flag=1
    elif (detector.numeroDedos()==2):
        # Cargar la imagen de las gafas
        img = cv2.imread('gafas2.jpg')
        flag=1
    elif (detector.numeroDedos()==0):
        flag=0

    if(flag==1):

        # Convertir las imágenes a escala de grises
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Aplicar un umbral para crear imágenes binarias
        _, umbral = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
        # Encontrar los contornos en la imagen binaria
        contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Crear una máscara vacía
        mascara = np.zeros_like(img)
        # Rellenar las máscaras con el contorno del objeto 
        cv2.drawContours(mascara, contornos, -1, (255, 255, 255), thickness=cv2.FILLED)
        # Extraer lel objeto (gafas)
        objeto_extraido = cv2.bitwise_and(img, mascara)
        
        # Convertir el fotograma a RGB para MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        #Detector de cara
        results = face_mesh.process(frame_rgb)
        results2 = face_detection.process(frame_rgb)

        #solo cordenadas de la fokin cabeza
        if results2.detections:
            for detection in results2.detections:
                # Obtener la caja delimitadora (bounding box) de la cara
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                
        # detector ojos y facciones dentro de la cara
        if results.multi_face_landmarks:
            for landmarks in results.multi_face_landmarks:
                # Obtener las coordenadas de los ojos izquierdo (punto 33) y derecho (punto 133)
                left_eye = landmarks.landmark[33]  # Ojo izquierdo
                right_eye = landmarks.landmark[133]  # Ojo derecho

                # Convertir las coordenadas normalizadas a píxeles
                left_eye_x = int(left_eye.x * iw)
                left_eye_y = int(left_eye.y * ih)
                right_eye_x = int(right_eye.x * iw)
                right_eye_y = int(right_eye.y * ih)
                        
                # Calcular el punto medio entre los ojos
                center_x = (left_eye_x + right_eye_x) // 2 + 35
                center_y = (left_eye_y + right_eye_y) // 2
                        
                # Establecer un factor de escala para las gafas
                escala_gafas = 1.2  # Ajustar el tamaño de las gafas
                overlay = cv2.resize(objeto_extraido, (int(w * escala_gafas), int(h * escala_gafas)))

                # Determinar la nueva posición para colocar las gafas centradas entre los ojos
                new_x = center_x - overlay.shape[1] // 2  # Centrado horizontalmente entre los ojos
                new_y = center_y - overlay.shape[0] // 2  # Centrado verticalmente entre los ojos

                # Asegurarse de que las gafas no se muevan fuera de los límites de la imagen
                new_x = max(new_x, 0)
                new_y = max(new_y, 0)

                # Obtener la región del fotograma donde se colocarán las gafas
                n_frame = frame[new_y:new_y+overlay.shape[0], new_x:new_x+overlay.shape[1]]
                
                # Convertir la imagen de las gafas a escala de grises para obtener la máscara
                gray_overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)
                        
                # Aplicar un umbral a la imagen en escala de grises para generar la máscara
                _, mask = cv2.threshold(gray_overlay, 1, 255, cv2.THRESH_BINARY)

                # Invertir la máscara
                mask_inv = cv2.bitwise_not(mask)
                        
                # Extraer la parte de la cara (fondo) que no tiene las gafas
                bg_frame = cv2.bitwise_and(n_frame, n_frame, mask=mask_inv)
                        
                # Extraer la parte de las gafas donde se va a colocar
                fg_overlay = cv2.bitwise_and(overlay, overlay, mask=mask)
                        
                # Combinar el fondo de la cara y las gafas en la misma región
                result = cv2.add(bg_frame, fg_overlay)
                        
                # Superponer el resultado en el fotograma original
                frame[new_y:new_y+overlay.shape[0], new_x:new_x+overlay.shape[1]] = result

    # Mostrar el fotograma con las gafas colocadas
    cv2.imshow('Frame', frame)
    
    # Detener la ejecución con la tecla ESC
    if cv2.waitKey(20) == 27:
        break

# Liberar la captura de video y cerrar las ventanas
vid.release()
cv2.destroyAllWindows()


I0000 00:00:1731592955.583133 2133096 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1731592955.723572 2133096 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
W0000 00:00:1731592955.733922 2141218 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731592955.740670 2141241 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731592955.755302 2141220 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1731592955.759975 2133096 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM)

In [ ]:
import warnings

detector = detectorManos(0.75)
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database') # Ignorar advertencia de protobuf

# Inicializar MediaPipe Face Mesh para detectar los puntos de la cara
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.2)
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.2)

# Iniciar la captura de video
vid = cv2.VideoCapture(0)

flag=0

while(True):
    
    ret, frame = vid.read()
    if not ret:
            break
    
    frame = detector.encontrarManos(frame)
    manosInfo, cuadro = detector.encontrarPosicion(frame, dibujar=False)
    
    if(detector.numeroDedos()== 1):
        # Cargar la imagen de las gafas
        img = cv2.imread('cara_ramon.jpg')
        flag=1
    elif (detector.numeroDedos()==2):
        # Cargar la imagen de las gafas
        img = cv2.imread('cara_noah.jpg')
        flag=1
    elif (detector.numeroDedos()==0):
        flag=0

    if(flag==1):

        # Convertir las imágenes a escala de grises
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Aplicar un umbral para crear imágenes binarias
        _, umbral = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
        # Encontrar los contornos en la imagen binaria
        contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Crear una máscara vacía
        mascara = np.zeros_like(img)
        # Rellenar las máscaras con el contorno del objeto 
        cv2.drawContours(mascara, contornos, -1, (255, 255, 255), thickness=cv2.FILLED)
        # Extraer lel objeto (gafas)
        objeto_extraido = cv2.bitwise_and(img, mascara)
        
        # Convertir el fotograma a RGB para MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        #Detector de cara
        results = face_mesh.process(frame_rgb)
        results2 = face_detection.process(frame_rgb)

        #solo cordenadas de la fokin cabeza
        if results2.detections:
            for detection in results2.detections:
                # Obtener la caja delimitadora (bounding box) de la cara
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                
        # detector ojos y facciones dentro de la cara
        if results2.detections:
        for detection in results2.detections:
            # Obtener la caja delimitadora (bounding box) de la cara
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
            
            # Calcular el centro de la cara usando el bounding box
            center_x = x + w // 2
            center_y = y + h // 2
            
            # Establecer un factor de escala para la imagen de la cara
            escala_cara = 1.2  # Ajustar el tamaño de la imagen de la cara
            overlay = cv2.resize(objeto_extraido, (int(w * escala_cara), int(h * escala_cara)))

            # Determinar la nueva posición para colocar la imagen centrada en la cara
            new_x = center_x - overlay.shape[1] // 2
            new_y = center_y - overlay.shape[0] // 2

            # Asegurarse de que la imagen no se mueva fuera de los límites de la imagen principal
            new_x = max(new_x, 0)
            new_y = max(new_y, 0)

            # Obtener la región del fotograma donde se colocará la imagen de la cara
            n_frame = frame[new_y:new_y+overlay.shape[0], new_x:new_x+overlay.shape[1]]
            
            # Convertir la imagen de la cara a escala de grises para obtener la máscara
            gray_overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)
                    
            # Aplicar un umbral a la imagen en escala de grises para generar la máscara
            _, mask = cv2.threshold(gray_overlay, 1, 255, cv2.THRESH_BINARY)

            # Invertir la máscara
            mask_inv = cv2.bitwise_not(mask)
                    
            # Extraer la parte de la cara (fondo) que no tiene la imagen sobrepuesta
            bg_frame = cv2.bitwise_and(n_frame, n_frame, mask=mask_inv)
                    
            # Extraer la parte de la imagen que se va a colocar
            fg_overlay = cv2.bitwise_and(overlay, overlay, mask=mask)
                    
            # Combinar el fondo de la cara y la imagen en la misma región
            result = cv2.add(bg_frame, fg_overlay)
                    
            # Superponer el resultado en el fotograma original
            frame[new_y:new_y+overlay.shape[0], new_x:new_x+overlay.shape[1]] = result

    # Mostrar el fotograma con las gafas colocadas
    cv2.imshow('Frame', frame)
    
    # Detener la ejecución con la tecla ESC
    if cv2.waitKey(20) == 27:
        break

# Liberar la captura de video y cerrar las ventanas
vid.release()
cv2.destroyAllWindows()


I0000 00:00:1731595026.989983 2189377 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
I0000 00:00:1731595027.000638 2189377 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
W0000 00:00:1731595027.007227 2212821 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731595027.015138 2212811 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1731595027.026763 2189377 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
W0000 00:00:1731595027.051562 2212829 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731595027.062248 2212811

error: OpenCV(4.10.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
